In [9]:
import pandas as pd
import scipy.io as sio
from scipy import signal, stats
import numpy as np
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import re
import coare36vn_zrf_et
import os

oceanDF = pd.read_csv('ocean_temperature_interpol12_13.csv', parse_dates = ['DN'])
oceanDF['hourDN'] = oceanDF['DN'].dt.floor('h')

hourlyMeanOceanDF = oceanDF.groupby('hourDN').agg(hourlyTemp=('OceanTemp', "mean")).reset_index()
hourlyMeanOceanDF['hourSalinity'] = oceanDF["oceanSalEstimate"]

heatFluxModelDF = pd.read_csv('ocean12_13_flux.csv', parse_dates = ['Tm (UTC)'], dayfirst = True)

heatFluxModelDF = pd.merge(left = hourlyMeanOceanDF, right = heatFluxModelDF, how = 'inner',  left_on = 'hourDN', right_on = 'Tm (UTC)')
heatFluxModelDF = heatFluxModelDF.drop('Tm (UTC)', axis = 1)
jdDateBase = pd.Timestamp('1858-11-17').to_julian_date()
heatFluxModelDF['jdTimeStamp'] = heatFluxModelDF['hourDN'].apply(lambda x: x.to_julian_date()- jdDateBase)
heatFluxModelDF = heatFluxModelDF.sort_values(['hourDN'], ascending = [True])
pd.set_option('display.float_format', '{:.20f}'.format)

A=coare36vn_zrf_et.coare36vn_zrf_et(heatFluxModelDF["U (m s-1)"].to_numpy(), 10, heatFluxModelDF["Tair (degC)"].to_numpy(),2, heatFluxModelDF["RH (%)"].to_numpy(),2, heatFluxModelDF["P (mb)"].to_numpy(), heatFluxModelDF["hourlyTemp"].to_numpy(), heatFluxModelDF["SW_dn (W m-2)"].to_numpy(), heatFluxModelDF["LW_dn (W m-2)"].to_numpy(), heatFluxModelDF["Lat"].to_numpy(), heatFluxModelDF["Lon"].to_numpy(),heatFluxModelDF["jdTimeStamp"].to_numpy(),600,0, heatFluxModelDF["hourSalinity"].to_numpy(),None,None,10,10,10)

fnameA = os.path.join("C:\\Users\\nvleb\\OneDrive - Oregon State University\\Desktop\\Python\\AlseaEstuaryData\\Alsea-Estuary-Data",'12_13_output_ocean_heatflux.csv')
# A=coare36vn_zrf_et(u, zu , t, zt, rh, zq, P, ts, sw_dn, lw_dn, lat, lon,jd, zi,rain, Ss, None , None, zrf_u, zrf_t, zrf_q)
# fnameA = os.path.join(path,'test_36_output_py_082022_withnowavesinput.txt')
A_hdr = 'usr\ttau\thsb\thlb\thbb\thsbb\thlwebb\ttsr\tqsr\tzo\tzot\tzoq\tCd\t'
A_hdr += 'Ch\tCe\tL\tzeta\tdT_skinx\tdq_skinx\tdz_skin\tUrf\tTrf\tQrf\t'
A_hdr += 'RHrf\tUrfN\tTrfN\tQrfN\tlw_net\tsw_net\tLe\trhoa\tUN\tU10\tU10N\t'
A_hdr += 'Cdn_10\tChn_10\tCen_10\thrain\tQs\tEvap\tT10\tT10N\tQ10\tQ10N\tRH10\t'
A_hdr += 'P10\trhoa10\tgust\twc_frac\tEdis'
csvHdr = A_hdr.replace("\t", ",")
np.savetxt(fnameA,A,fmt='%.18e',delimiter=',',header=csvHdr)

processedDataDF = pd.read_csv('12_13_output_ocean_heatflux.csv')
processedDataDF['Tm (UTC)'] = heatFluxModelDF['hourDN']

processedDataDF["Qnet"] = processedDataDF["sw_net"] + processedDataDF["lw_net"] - processedDataDF["hsb"] - processedDataDF["hlb"]
print(processedDataDF["Qnet"], processedDataDF["Tm (UTC)"])


c:\Users\nvleb\OneDrive - Oregon State University\Desktop\Python\AlseaEstuaryData\Alsea-Estuary-Data\coare36vn_zrf_et.py:750: RuntimeWarning: invalid value encountered in power
  psi = - ((1 + 0.6667 * zeta) ** 1.5 + np.multiply(0.6667 * (zeta - 14.28),np.exp(- dzeta)) + 8.525)
c:\Users\nvleb\OneDrive - Oregon State University\Desktop\Python\AlseaEstuaryData\Alsea-Estuary-Data\coare36vn_zrf_et.py:715: RuntimeWarning: invalid value encountered in power
  wc_frac = 0.00073 * (U10N - 2) ** 1.43


0      -72.12956455320993143232
1      -18.14087851492238101514
2       32.75409150523643120323
3       26.30210647297842285752
4      105.81550831686001856724
                 ...           
3838   -36.10983550351426885072
3839    -9.10771432799323221730
3840    11.33768146709272528483
3841     5.89570583314106499984
3842   -26.86538754414922181013
Name: Qnet, Length: 3843, dtype: float64 0      2012-12-12 17:00:00
1      2012-12-12 18:00:00
2      2012-12-12 19:00:00
3      2012-12-12 20:00:00
4      2012-12-12 21:00:00
               ...        
3838   2013-05-21 15:00:00
3839   2013-05-21 16:00:00
3840   2013-05-21 17:00:00
3841   2013-05-21 18:00:00
3842   2013-05-21 19:00:00
Name: Tm (UTC), Length: 3843, dtype: datetime64[ns]
